In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/chatbot deva2/nexyug_chatbot_dataset_30001.xlsx')

In [4]:
df.count()

,0
User Input,3000
Bot Response,3000
Intent,3000


In [5]:
df = df.dropna(subset=['User Input', 'Intent','Bot Response'])

In [6]:
df.isnull().sum()

,0
User Input,0
Bot Response,0
Intent,0


In [7]:
df["User Input"]=df["User Input"].str.lower()
df["Bot Response"]=df["Bot Response"].str.lower()
df["Intent"]=df["Intent"].str.lower()
df.head(2)

,User Input,Bot Response,Intent
0,goodbye #1,goodbye! stay safe.,goodbye
1,what is your mission? #2,our mission is to innovate and deliver top ai ...,company_info


In [8]:
pattern_to_remove = r'#\d+'
df["User Input"]=df["User Input"].str.replace(pattern_to_remove, '', regex=True)
df["User Input"]=df["User Input"].str.replace(r'[^\w\s]', '', regex=True)
df.head(2)

,User Input,Bot Response,Intent
0,goodbye,goodbye! stay safe.,goodbye
1,what is your mission,our mission is to innovate and deliver top ai ...,company_info


In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
df['User Input'] = df['User Input'].fillna('').astype(str)
df['tokenized_user_input'] = df['User Input'].apply(word_tokenize)

In [11]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords_from_list(tokens):
    return [word for word in tokens if word not in stop_words]

df['filtered_user_input'] = df['tokenized_user_input'].apply(remove_stopwords_from_list)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,User Input,Bot Response,Intent,tokenized_user_input,filtered_user_input
0,goodbye,goodbye! stay safe.,goodbye,[goodbye],[goodbye]
1,what is your mission,our mission is to innovate and deliver top ai ...,company_info,"[what, is, your, mission]",[mission]
2,thanks a lot,anytime!,thank_you,"[thanks, a, lot]","[thanks, lot]"
3,how many employees do you have,we have over 200 employees globally.,company_info,"[how, many, employees, do, you, have]","[many, employees]"
4,bye,goodbye! have a great day!,goodbye,[bye],[bye]


In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['encoded_intent'] = label_encoder.fit_transform(df['Intent'])
df[['Intent', 'encoded_intent']]

,Intent,encoded_intent
0,goodbye,1
1,company_info,0
2,thank_you,6
3,company_info,0
4,goodbye,1
...,...,...
2995,services_inquiry,4
2996,pricing,3
2997,services_inquiry,4
2998,services_inquiry,4


In [14]:
dataset = []

for _, row in df.iterrows():
    user_input = " ".join(row['filtered_user_input'])  # no eval needed
    response = row['Bot Response']
    dataset.append({
        "input": user_input,
        "output": response
    })


In [16]:
import json
with open("chatbot_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [25]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 requires this

# Load your JSON dataset
df = pd.read_json("/content/chatbot_data.json")
dataset = Dataset.from_pandas(df)

# Tokenize function
def tokenize_function(example):
    prompt = f"User: {example['input']}\nBot:"
    full_text = prompt + " " + example["output"]

    tokens = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [26]:
tokenized = dataset.map(tokenize_function, remove_columns=dataset.column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [27]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True  # if you're using GPU
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-27-30e0c3e7498c>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mokshbhardwaj2333 (mokshbhardwaj2333-instagram) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,0.237700
1000,0.030900
1500,0.029500
2000,0.029000
2500,0.028600


Step,Training Loss
500,0.237700
1000,0.030900
1500,0.029500
2000,0.029000
2500,0.028600
3000,0.028500


TrainOutput(global_step=3000, training_loss=0.06404086716969808, metrics={'train_runtime': 521.866, 'train_samples_per_second': 22.994, 'train_steps_per_second': 5.749, 'total_flos': 783876096000000.0, 'train_loss': 0.06404086716969808, 'epoch': 4.0})

In [30]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [31]:
def generate_response(user_input):
    prompt = f"User: {user_input}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # ✅ move to GPU if available

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Bot:")[-1].strip()


In [46]:
print(generate_response("I need help with my account"))

you can email support@nexyugtech.com or call +1-800-555-0199.


In [47]:
model.save_pretrained("gpt2-chatbot-model")
tokenizer.save_pretrained("gpt2-chatbot-model")

('gpt2-chatbot-model/tokenizer_config.json',
 'gpt2-chatbot-model/special_tokens_map.json',
 'gpt2-chatbot-model/vocab.json',
 'gpt2-chatbot-model/merges.txt',
 'gpt2-chatbot-model/added_tokens.json',
 'gpt2-chatbot-model/tokenizer.json')

In [48]:
!zip -r gpt2-chatbot-model.zip gpt2-chatbot-model

  adding: gpt2-chatbot-model/ (stored 0%)
  adding: gpt2-chatbot-model/config.json (deflated 51%)
  adding: gpt2-chatbot-model/special_tokens_map.json (deflated 60%)
  adding: gpt2-chatbot-model/tokenizer.json (deflated 82%)
  adding: gpt2-chatbot-model/merges.txt (deflated 53%)
  adding: gpt2-chatbot-model/tokenizer_config.json (deflated 54%)
  adding: gpt2-chatbot-model/generation_config.json (deflated 24%)
  adding: gpt2-chatbot-model/vocab.json (deflated 59%)
  adding: gpt2-chatbot-model/model.safetensors (deflated 7%)


In [49]:
from google.colab import files
files.download("gpt2-chatbot-model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>